# Integrating Phenotypic Causal Networks in GWAS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QAHRoddur/JWAS/blob/main/Examples/integrating_phenotypic_causal_networks_in_gwas.ipynb)

This notebook is auto-generated from the JWAS.jl wiki page.


In [ ]:
using Pkg
Pkg.add("JWAS")
Pkg.precompile()
using JWAS


In [ ]:
# One-time setup each Colab session
if !isdir("/content/JWAS")
    run(`git clone https://github.com/QAHRoddur/JWAS.git /content/JWAS`)
end
cd("/content/JWAS/Examples")
pwd()  # should show /content/JWAS/Examples


# Structure Equation Models for GWAS


In [ ]:
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets

# Step 2: Read data 
phenofile  = "../data/phenotypes.txt"
pedfile    = "../data/pedigree.txt"
genofile   = "../data/genotypes.txt"
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstring=["NA"])
pedigree   = get_pedigree(pedfile,separator=",",header=true);
genotypes  = get_genotypes(genofile,separator=',',method="BayesC");
first(phenotypes,5)

# Step 3: Build Model Equations

model_equation  ="y1 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes
                  y2 = intercept + x1 + x2 + ID + genotypes
                  y3 = intercept + x1 + ID + genotypes";
model = build_model(model_equation);

# Step 4: Set Factors or Covariates
set_covariate(model,"x1");

# Step 5: Set Random or Fixed Effects
set_random(model,"x2");
set_random(model,"ID dam",pedigree);

# Step 6: Run Analysis
# If `causal_structure` is provided, e.g., causal_structure = [0.0 0.0 0.0;1.0 0.0 0.0;1.0 0.0 0.0] (column index affects row index, and a lower triangular matrix is required) for
# trait 1 -> trait 2 and trait 1 -> trait 3, phenotypic causal networks will be incorporated using structure equation models.
my_structure = [0.0 0.0 0.0
                1.0 0.0 0.0
                1.0 0.0 0.0]
out=runMCMC(model,phenotypes,causal_structure=my_structure);


# GWAS of Direct Marker Effects on Trait y2


In [ ]:
#Compute the model frequency for each marker (the probability the marker is included in the model).
marker_effects_file="results/MCMC_samples_marker_effects_genotypes_y2.txt"
GWAS(marker_effects_file,header=true)

#Compute the posterior probability of association of the genomic window that explains a large proportion of the total genetic variance
map_file="../data/map.txt"
marker_effects_file="results/MCMC_samples_marker_effects_genotypes_y2.txt"
out=GWAS(model,map_file,marker_effects_file,header=true,window_size="1 Mb")


# GWAS of Indirect Marker Effects on Trait y2


In [ ]:
#Compute the model frequency for each marker (the probability the marker is included in the model).
marker_effects_file="results/MCMC_samples_indirect_marker_effects_genotypes_y2.txt"
GWAS(marker_effects_file,header=true)

#Compute the posterior probability of association of the genomic window that explains a large proportion of the total genetic variance
map_file="../data/map.txt"
marker_effects_file="results/MCMC_samples_indirect_marker_effects_genotypes_y2.txt"
out=GWAS(model,map_file,marker_effects_file,header=true,window_size="1 Mb")
